# 금통위 의사록 데이터 전처리

## 단계 1. 금통위 의사록 섹션 분리

KoNLPY 설치

In [ ]:
!sudo apt-get install g++ openjdk-7-jdk

In [ ]:
!sudo apt-get install python-dev; pip install konlpy

In [ ]:
 !sudo apt-get install python3-dev; pip3 install konlpy   

mecab 설치

In [ ]:
!sudo apt-get install curl

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


eKoNLPY 설치

In [ ]:
!git clone https://github.com/entelecheia/eKoNLPy.git

In [ ]:
cd eKoNLPy

In [ ]:
!pip install .

In [ ]:
!pip install . --upgrade

금통위 의사록 섹션 분리 -> csv 파일로 저장

In [1]:
pip install html2text

In [22]:
import html2text
import re
from datetime import datetime, timedelta

In [44]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [45]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [46]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text) :
    mpck = MPCK()
    tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(tokens)
                
    return tokens+ ngrams

In [47]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join

    
def preprocessing(source_folder="./text/", output_file="bok_minutes_final.csv") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    txt_files.sort()
    df = pd.DataFrame(columns=['date', 'minutes'])
    df["Economic Situation"] = ""
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df["Monetary Policy"] = ""
    df["Participant Views"] = ""
    df["Government View"] = ""

    df["Economic Situation count"] = ""
    df["Foreign Currency count"] = ""
    df["Financial Markets count"] = ""
    df["Monetary Policy count"] = ""
    df["Participant Views count"] = ""
    df["Government View count"] = ""

    df["ngram"] = ""
    

    for txt_file in txt_files :
        with open(source_folder + txt_file, 'r') as f :
            txt = f.read()
            sections, section_texts = preprocess_minutes(txt)
            
            print(txt_file)
            
            ngrams = []
            for section_text in section_texts[1:3] :
                for sentence in section_text :
                    ngrams.append(','.join(text2ngram(sentence)))
                
            #print('@@@'.join(ngrams))
            #break

            df.loc[len(df)] = [txt_file.split('_')[0],
                               txt,
                               '@@@'.join(section_texts[0]),
                               '@@@'.join(section_texts[1]),
                               '@@@'.join(section_texts[2]),
                               '@@@'.join(section_texts[3]),
                               '@@@'.join(section_texts[4]),
                               '@@@'.join(section_texts[5]),
                               len(section_texts[0]),
                               len(section_texts[1]),
                               len(section_texts[2]),
                               len(section_texts[3]),
                               len(section_texts[4]),
                               len(section_texts[5]),
                               '@@@'.join(ngrams)]
    
    df.to_csv(output_file, index=False)
        
    print(df.shape)
preprocessing()

금융통화위원회 의사록(2005년도 제12차)(2005.6.9).txt
금융통화위원회 의사록(2005년도 제13차)(2005.6.23).txt
금융통화위원회 의사록(2005년도 제14차)(2005.7.7).txt
금융통화위원회 의사록(2005년도 제17차)(2005.8.11).txt
금융통화위원회 의사록(2005년도 제19차)(2005.9.8).txt
금융통화위원회 의사록(2005년도 제20차)(2005.9.22).txt
금융통화위원회 의사록(2005년도 제21차)(2005.10.11).txt
금융통화위원회 의사록(2005년도 제24차)(2005.11.10).txt
금융통화위원회 의사록(2005년도 제26차)(2005.12.8).txt
금융통화위원회 의사록(2005년도 제27차)(2005.12.22).txt
금융통화위원회 의사록(2006년도 제10차)(2006.5.11).txt
금융통화위원회 의사록(2006년도 제11차)(2006.5.25).txt
금융통화위원회 의사록(2006년도 제12차)(2006.6.8).txt
금융통화위원회 의사록(2006년도 제13차)(2006.6.22).txt
금융통화위원회 의사록(2006년도 제14차)(2006.7.7).txt
금융통화위원회 의사록(2006년도 제16차)(2006.8.10).txt
금융통화위원회 의사록(2006년도 제17ᄎ

In [56]:
import pandas as pd
df = pd.read_csv("./bok_minutes_final.csv")
df['ngram'][0]

'경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없/VA,불구/NNG,원엔/NNG,환율/NNG,지속/NNG,로/NNG,하락/NNG,하/XSV,이유/NNG,물/VV,대해/VV,위안/NNG,절/NNG,상시/NNG,권/NNG,통화/NNG,절상/NNG,크/VA,기대/NNG,원화/NNG,달러/NNG,엔달러/NNG,환율/NNG,상승/NNG,추수/NNG,하/XSV,못하/VX,외환/NNG,수급/NNG,면/NNG,공급우위/NNG,상태/NNG,지속/NNG,되/XSV,국제투자은행/NNG,엔달러/NNG,환율/NNG,전망/NNG,상향/NNG,하/XSV,불구/NNG,원달러/NNG,조정/NNG,아직/MAG,이루/VV,않/VX,기업/NNG,기대/NNG,원화/NNG,절상/NNG,편향/NNG,되/XSV,기인/NNG,하/VV,설명/NNG,하/XSV,엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/NNG;절상/NNG,전망/NNG;상향/NNG@@@환율/NNG,장기/NNG,각국/NNG,기초/NNG,경제/NNG,여건/NNG,반/NNG,하/VV,움직이/VV,점/NNG,감안/NNG,하/VV,때/NNG,원엔/NNG,환율/NNG,하락/NNG,일시적/VAX,현상/NNG,장기/NNG,반전/NNG,되/VV,가능성/NNG,있/VA,보인다는/VV,견해/NNG,밝혔으며/VV,대해/VV,견해/NNG,동의/NNG,하/XSV,시장/NNG,규모/NNG,작/VA,일시적/VAX,외부충격/NNG,의하/VV,쉽/VA,받/VV,점/NNG,설명/NNG,하/XSV,내외/NNG,금리/NNG,역전/NNG,나타나/VV,같/VA,대규모/NNG,외환/NNG,유입/NNG,일어나/VV,않/VX,감안/NNG,하/VV,때/NNG,원엔/NNG,환율/NNG,추가/NNG,하락/NNG,하/VV,점차/MAG,축소/NNG,되/VV,생각/NNG,되/VV,답변/NNG,하/XSV,원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일시적/VAX@@@유로존/NNG,경제/NNG